In [3]:
import warnings
from sklearn.exceptions import ConvergenceWarning
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=ConvergenceWarning)

import pandas
import csv
import matplotlib.pyplot as plt

#preprocesamiento
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split

from numpy import mean
from numpy import std
import numpy as np
seed = 7
np.random.seed(seed)

# modelos
from sklearn.decomposition import PCA
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier

from sklearn.ensemble import StackingClassifier

#guardar modelos
from sklearn.externals import joblib

#evaluacion
from sklearn.metrics import *
path= "dataRegularizada/"

### Una vez preprocesado el dataset se debe ejecutar Training_Test_Data.py para crear 10 datasets para entrenamientos y test de cada modelo.

# Carga de datos

In [4]:
train_folds  = []
test_folds = []
for i in range(10):
    train = pandas.read_csv(path + "train_"+ str(i) + ".csv", decimal=".")
    col_mask=train.isnull().any(axis=0) 
    row_mask=train.isnull().any(axis=1)
    train_folds.append(train.loc[-row_mask,-col_mask])
    test = pandas.read_csv(path + "test_"+ str(i) + ".csv", decimal=".")
    col_mask=test.isnull().any(axis=0) 
    row_mask=test.isnull().any(axis=1)
    test_folds.append(test.loc[-row_mask,-col_mask])

In [3]:
train_folds[0]

,age,creatinine_apache,hematocrit_apache,map_apache,sodium_apache,temp_apache,wbc_apache,aids,cirrhosis,diabetes_mellitus,hepatic_failure,immunosuppression,leukemia,lymphoma,solid_tumor_with_metastasis,hospital_death
0,62.000000,0.70,27.200000,146.000000,136.0,38.90,18.40,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
1,63.000000,0.98,32.988739,43.000000,138.0,36.90,10.40,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
2,35.000000,0.40,36.700000,122.000000,132.0,37.00,17.20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
3,22.000000,0.98,32.988739,107.000000,138.0,36.33,10.40,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
4,80.000000,1.70,29.900000,200.000000,153.0,36.40,14.20,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0
5,59.000000,0.30,34.900000,63.000000,137.0,36.39,9.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
6,71.000000,0.98,32.988739,114.000000,138.0,36.60,10.40,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0
7,50.000000,1.19,34.700000,120.000000,135.0,39.10,9.89,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
8,34.000000,2.10,37.400000,53.000000,128.0,36.20,7.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
9,62.309516,0.98,32.988739,54.000000,138.0,36.50,10.40,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0


# Cost-sensitive  Naive Classifier

In [6]:
# fuente  -> https://machinelearningmastery.com/naive-classifiers-imbalanced-classification-metrics/
model_NC = []

for i in range(len(train_folds)):
    model =  GaussianNB()
    model.fit(train_folds[i].drop(columns=["hospital_death"]), train_folds[i]["hospital_death"])
    joblib.dump(model, path + 'NC'+ str(i) +'.pkl') 
    model_NC.append(model)
    predict = model.predict(test_folds[i].drop(columns=["hospital_death"]))
    result = classification_report(test_folds[i]["hospital_death"], predict, output_dict=True)
    recall = result["1"]["recall"]
    recalln = result["0"]["recall"]
    print("Recall Naives " + str(i) + ": Positive-> " + str(recall) +"\t Negative-> " + str(recalln))

Recall Naives 0: Positive-> 0.5517676767676768	 Negative-> 0.8920047732696897
Recall Naives 1: Positive-> 0.5441919191919192	 Negative-> 0.8788782816229117
Recall Naives 2: Positive-> 0.6426767676767676	 Negative-> 0.8690930787589499
Recall Naives 3: Positive-> 0.6060606060606061	 Negative-> 0.8507159904534606
Recall Naives 4: Positive-> 0.6502525252525253	 Negative-> 0.8332935560859188
Recall Naives 5: Positive-> 0.5524652338811631	 Negative-> 0.8853221957040572
Recall Naives 6: Positive-> 0.5802781289506953	 Negative-> 0.8729116945107399
Recall Naives 7: Positive-> 0.5916561314791403	 Negative-> 0.8805489260143198
Recall Naives 8: Positive-> 0.5600505689001264	 Negative-> 0.8812507459124
Recall Naives 9: Positive-> 0.5815423514538559	 Negative-> 0.8606038906790786


# Cost-sensitive  RANDOM FOREST

In [8]:
model_RF = []

for i in range(len(train_folds)):
    model = RandomForestClassifier(max_depth=3, random_state=0, class_weight='balanced', n_estimators=50)
    d = (train_folds[i]).drop(columns=["hospital_death"])
    model.fit(d, train_folds[i]["hospital_death"])
    joblib.dump(model, path + 'RF'+ str(i) +'.pkl') 
    model_RF.append(model)
    predict = model.predict(test_folds[i].drop(columns=["hospital_death"]))
    result = classification_report(test_folds[i]["hospital_death"], predict, output_dict=True)
    recall = result["1"]["recall"]
    recalln = result["0"]["recall"]
    print("Recall RF " + str(i) + ": Positive-> " + str(recall) +"\t Negative-> " + str(recalln))

Recall RF 0: Positive-> 0.7045454545454546	 Negative-> 0.8556085918854416
Recall RF 1: Positive-> 0.7411616161616161	 Negative-> 0.7910501193317423
Recall RF 2: Positive-> 0.76010101010101	 Negative-> 0.8014319809069212
Recall RF 3: Positive-> 0.7904040404040404	 Negative-> 0.7608591885441528
Recall RF 4: Positive-> 0.7828282828282829	 Negative-> 0.7786396181384249
Recall RF 5: Positive-> 0.7269279393173198	 Negative-> 0.8176610978520287
Recall RF 6: Positive-> 0.7168141592920354	 Negative-> 0.8122911694510739
Recall RF 7: Positive-> 0.7243994943109987	 Negative-> 0.7976133651551313
Recall RF 8: Positive-> 0.706700379266751	 Negative-> 0.8045112781954887
Recall RF 9: Positive-> 0.7269279393173198	 Negative-> 0.8024823964673589


# Cost-sensitive  SUPPORT VECTOR MACHINE

In [13]:
#fuente ->  https://machinelearningmastery.com/cost-sensitive-svm-for-imbalanced-classification/
model_SVM = []

for i in range(len(train_folds)):
    model = SVC(gamma='scale', class_weight='balanced')
    model.fit(train_folds[i].drop(columns=["hospital_death"]), train_folds[i]["hospital_death"])
    joblib.dump(model, path + 'SVM'+ str(i) +'.pkl') 
    model_SVM.append(model)
    predict = model.predict(test_folds[i].drop(columns=["hospital_death"]))
    result = classification_report(test_folds[i]["hospital_death"], predict, output_dict=True)
    recall = result["1"]["recall"]
    recalln = result["0"]["recall"]
    print("Recall SVM " + str(i) + ": Positive-> " + str(recall) +"\t Negative-> " + str(recalln))

Recall SVM 0: Positive-> 0.7222222222222222	 Negative-> 0.8305489260143198
Recall SVM 1: Positive-> 0.7626262626262627	 Negative-> 0.7978520286396181
Recall SVM 2: Positive-> 0.8169191919191919	 Negative-> 0.7606205250596658
Recall SVM 3: Positive-> 0.7866161616161617	 Negative-> 0.7513126491646778
Recall SVM 4: Positive-> 0.7916666666666666	 Negative-> 0.7523866348448688
Recall SVM 5: Positive-> 0.7395701643489254	 Negative-> 0.8140811455847256
Recall SVM 6: Positive-> 0.7610619469026548	 Negative-> 0.789618138424821
Recall SVM 7: Positive-> 0.7585335018963337	 Negative-> 0.778400954653938
Recall SVM 8: Positive-> 0.7256637168141593	 Negative-> 0.789234992242511
Recall SVM 9: Positive-> 0.7926675094816688	 Negative-> 0.7511636233440745


# Cost-sensitive  LOGISTIC REGRESSION

In [10]:
# fuente -> https://machinelearningmastery.com/cost-sensitive-logistic-regression/
model_LR = []

model = LogisticRegression(solver='lbfgs', class_weight='balanced')
for i in range(len(train_folds)):
    model.fit(train_folds[i].drop(columns=["hospital_death"]), train_folds[i]["hospital_death"])
    joblib.dump(model,path +  'LR'+ str(i) +'.pkl')
    model_LR.append(model)
    predict = model.predict(test_folds[i].drop(columns=["hospital_death"]))
    result = classification_report(test_folds[i]["hospital_death"], predict, output_dict=True)
    recall = result["1"]["recall"]
    recalln = result["0"]["recall"]
    print("Recall LR " + str(i) + ": Positive-> " + str(recall) +"\t Negative-> " + str(recalln))

Recall LR 0: Positive-> 0.6893939393939394	 Negative-> 0.8305489260143198
Recall LR 1: Positive-> 0.7184343434343434	 Negative-> 0.802744630071599
Recall LR 2: Positive-> 0.8131313131313131	 Negative-> 0.7443914081145585
Recall LR 3: Positive-> 0.7676767676767676	 Negative-> 0.7402147971360382
Recall LR 4: Positive-> 0.7790404040404041	 Negative-> 0.7522673031026254
Recall LR 5: Positive-> 0.7029077117572693	 Negative-> 0.815035799522673
Recall LR 6: Positive-> 0.718078381795196	 Negative-> 0.8038186157517899
Recall LR 7: Positive-> 0.7458912768647282	 Negative-> 0.8009546539379475
Recall LR 8: Positive-> 0.6978508217446271	 Negative-> 0.7912638739706409
Recall LR 9: Positive-> 0.7737041719342604	 Negative-> 0.7385129490392648


# Decision Tree Classifier

In [11]:
model_DT= []

for i in range(len(train_folds)):
    model = DecisionTreeClassifier(class_weight='balanced')
    model.fit(train_folds[i].drop(columns=["hospital_death"]), train_folds[i]["hospital_death"])
    joblib.dump(model, path +  'DT'+ str(i) +'.pkl') 
    model_DT.append(model)
    predict = model.predict(test_folds[i].drop(columns=["hospital_death"]))
    result = classification_report(test_folds[i]["hospital_death"], predict, output_dict=True)
    recall = result["1"]["recall"]
    recalln = result["0"]["recall"]
    print("Recall TC " + str(i) + ": Positive-> " + str(recall) +"\t Negative-> " + str(recalln))

Recall TC 0: Positive-> 0.2828282828282828	 Negative-> 0.9478520286396181
Recall TC 1: Positive-> 0.30934343434343436	 Negative-> 0.9326968973747016
Recall TC 2: Positive-> 0.3383838383838384	 Negative-> 0.9353221957040573
Recall TC 3: Positive-> 0.3522727272727273	 Negative-> 0.9278042959427207
Recall TC 4: Positive-> 0.3055555555555556	 Negative-> 0.9247016706443915
Recall TC 5: Positive-> 0.3147914032869785	 Negative-> 0.9353221957040573
Recall TC 6: Positive-> 0.27560050568900124	 Negative-> 0.9445107398568019
Recall TC 7: Positive-> 0.26548672566371684	 Negative-> 0.9313842482100239
Recall TC 8: Positive-> 0.2857142857142857	 Negative-> 0.9393722401241198
Recall TC 9: Positive-> 0.32616940581542353	 Negative-> 0.9336436328917532


# knn

In [12]:
model_KKN = []
recall_KKN = []

for i in range(len(train_folds)):
    model = KNeighborsClassifier(n_neighbors=30)
    n_neighbors = range(1, 10, 3)
    weights = ['uniform', 'distance']
    metric = ['euclidean', 'manhattan', 'minkowski']
    #grid = dict(n_neighbors=n_neighbors,weights=weights,metric=metric)
    #grid_search = GridSearchCV(estimator=model, param_grid=grid, n_jobs=-1, scoring='recall',error_score=0)
    #grid_result = grid_search.fit(train_folds[i].drop(columns=["hospital_death"]), train_folds[i]["hospital_death"])
    model.fit(train_folds[i].drop(columns=["hospital_death"]), train_folds[i]["hospital_death"])
    model_KKN.append(model)
    joblib.dump(model,path +   'KNN'+ str(i) +'.pkl') 
    predict = model.predict(test_folds[i].drop(columns=["hospital_death"]))
    result = classification_report(test_folds[i]["hospital_death"], predict, output_dict=True)
    recall = result["1"]["recall"]
    recalln = result["0"]["recall"]
    print("Recall Knn " + str(i) + ": Positive-> " + str(recall) +"\t Negative-> " + str(recalln))
    # summarize results
    """
    print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
    means = grid_result.cv_results_['mean_test_score']
    stds = grid_result.cv_results_['std_test_score']
    params = grid_result.cv_results_['params']
    for mean, stdev, param in zip(means, stds, params):
        print("%f (%f) with: %r" % (mean, stdev, param))
    """

Recall Knn 0: Positive-> 0.05176767676767677	 Negative-> 0.9992840095465394
Recall Knn 1: Positive-> 0.03409090909090909	 Negative-> 0.9992840095465394
Recall Knn 2: Positive-> 0.016414141414141416	 Negative-> 1.0
Recall Knn 3: Positive-> 0.05555555555555555	 Negative-> 0.9992840095465394
Recall Knn 4: Positive-> 0.025252525252525252	 Negative-> 0.999164677804296
Recall Knn 5: Positive-> 0.05562579013906448	 Negative-> 0.9989260143198091
Recall Knn 6: Positive-> 0.051833122629582805	 Negative-> 0.9985680190930788
Recall Knn 7: Positive-> 0.0316055625790139	 Negative-> 0.9992840095465394
Recall Knn 8: Positive-> 0.0316055625790139	 Negative-> 0.9995226160639694
Recall Knn 9: Positive-> 0.0695322376738306	 Negative-> 0.998687194175916


# Stochastic Gradient Boosting

In [ ]:
model_SGB = []
model = GradientBoostingClassifier()

for i in range(len(train_folds)):
    model.fit(train_folds[i].drop(columns=["hospital_death"]), train_folds[i]["hospital_death"])
    joblib.dump(model, path + 'SGB'+ str(i) +'.pkl') 
    model_SGB.append(model)

# Cost-Sensitive Neural Network

In [48]:
# https://machinelearningmastery.com/tutorial-first-neural-network-python-keras/
# https://machinelearningmastery.com/grid-search-hyperparameters-deep-learning-models-python-keras/
# https://machinelearningmastery.com/cost-sensitive-neural-network-for-imbalanced-classification/

In [15]:
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras import backend as K

def recall_m(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

def create_model(optimizer='adam', neurons=80):
    # define the keras model
    model = Sequential()
    model.add(Dense(180, input_dim=15, activation='relu'))
    model.add(Dense(neurons, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    # compile the keras model
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=["accuracy"])
    return model

Using TensorFlow backend.


In [10]:
model_NR = []
recall_NR = []
model = KerasClassifier(build_fn=create_model, verbose=0)
batch_size = [2000]
neurons = [60,80]
epochs = [20]
param_grid = dict(batch_size=batch_size, epochs=epochs, neurons=neurons)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)
for i in range(len(train_folds)):
    grid_result = grid.fit(train_folds[i].drop(columns=["hospital_death"]), train_folds[i]["hospital_death"], class_weight='balanced')
    # summarize results
    predict = grid_result.predict(test_folds[i].drop(columns=["hospital_death"]))
    recall = recall_score(test_folds[i]["hospital_death"], predict, average='binary')
    print('Recall %d fold: %f' % (i , recall))
    recall_NR.append(recall)
    print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
    means = grid_result.cv_results_['mean_test_score']
    stds = grid_result.cv_results_['std_test_score']
    params = grid_result.cv_results_['params']
    for mean, stdev, param in zip(means, stds, params):
        print("%f (%f) with: %r" % (mean, stdev, param))






Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

Recall 0 fold: 0.111111
Best: 0.915909 using {'batch_size': 2000, 'neurons': 80, 'epochs': 20}
0.915896 (0.008760) with: {'batch_size': 2000, 'neurons': 60, 'epochs': 20}
0.915909 (0.008865) with: {'batch_size': 2000, 'neurons': 80, 'epochs': 20}
Recall 1 fold: 0.070707
Best: 0.916030 using {'batch_size': 2000, 'neurons': 80, 'epochs': 20}
0.915884 (0.008820) with: {'batch_size': 2000, 'neurons': 60, 'epochs': 20}
0.916030 (0.008889) with: {'batch_size': 2000, 'neurons': 80, 'epochs': 20}
Recall 2 fold: 0.089646
Best: 0.915896 using {'batch_size': 2000, 'neurons': 80, 'epochs': 20}
0.915448 (0.008321) with: {'batch_size': 2000, 'neurons': 60, 'epochs': 20}
0.915896 (0.009117) with: {'batch_size': 2000, 'neurons': 80, 'epochs': 20}
Recall 3 fold: 0.080910
Best: 0.916006 using {'batch_size': 2000, 'neurons': 80, 'epochs': 20}
0.915885 (0.007049) with: {'batch_size': 2000, 'neurons': 60, 'e

In [17]:
model_SVM = []
for i in range(len(train_folds)):
    model = joblib.load(path + 'SVM' + str(i) + '.pkl')
    model_SVM.append(model)

# Stacking voting

In [ ]:
model_SC = []

for i in range(len(train_folds)):
    estimators = [ ('nv', model_NC[i]), ('rf', model_RF[i]), ('svm',model_SVM[i]), ('lr',model_LR[i]), ('dt',model_DT[i]), ('knn',model_KKN[i]), ('sgb',model_SGB[i])]
    model = clf = StackingClassifier(estimators=estimators, final_estimator=LogisticRegression())
    model.fit(train_folds[i].drop(columns=["hospital_death"]), train_folds[i]["hospital_death"])
    joblib.dump(model,path +  'stack'+ str(i) +'.pkl') 
    model_SC.append(model)
    print("Modelo stack: " + str(i) + "guardado")